In [1]:
import numpy as np
from numpy import linalg as LA
import pickle
import sys
import os
from keras.models import load_model
from scipy.linalg import eigh

def cosine(x):
    d_inner = np.dot(x.T,x) #inner product

    l2 = np.linalg.norm(x,axis=0)
    d_norm = np.outer(l2.T,l2) #l2 norm
    d_norm = (d_inner+0.00001)/(d_norm+0.00001)
    return d_norm 

import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
session = tf.Session(config=config)

fcn1_initial = load_model('relu_cifar100_5000-2500-2500-1250/initial.h5')
fcn1_final = load_model('relu_cifar100_5000-2500-2500-1250'+'/final.h5')
print(1)
fcn2_initial = load_model('relu_cifar100_10000-1000-1000-1000/initial.h5')
fcn2_final = load_model('relu_cifar100_10000-1000-1000-1000'+'/final.h5')
print(2)
vgg11_initial = load_model('vgg11_cifar100/initial.h5')
vgg11_final = load_model('vgg11_cifar100'+'/final.h5')
print(3)
vgg16_initial = load_model('vgg16_cifar100/initial.h5')
vgg16_final = load_model('vgg16_cifar100'+'/final.h5')
print(4)
vgg19_initial = load_model('vgg19_cifar100/initial.h5')
vgg19_final = load_model('vgg19_cifar100'+'/final.h5')
print(5)
resnet18_initial = load_model('resnet18_cifar100/initial.h5')
resnet18_final = load_model('resnet18_cifar100'+'/final.h5')
print(6)
resnet34_initial = load_model('resnet34_cifar100/initial.h5')
resnet34_final = load_model('resnet34_cifar100'+'/final.h5')
print(7)
resnet50_initial = load_model('resnet50_cifar100/initial.h5')
resnet50_final = load_model('resnet50_cifar100'+'/final.h5')
print(8)
densenet121_initial = load_model('densenet121_cifar100/initial.h5')
densenet121_final = load_model('densenet121_cifar100'+'/final.h5')
print(9)

Using TensorFlow backend.
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflo

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
1
2
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
3
4
5
6
7
8
9


In [2]:
import sys 
sys.path.append("..") 
import utils

gap = []
trn, tst = utils.get_cifar100('FCN')
for z in [fcn1_final, fcn2_final]:
    trnloss, trnacc = z.evaluate(trn.X, trn.Y, verbose=0)
    tstloss, tstacc = z.evaluate(tst.X, tst.Y, verbose=0)
    print(tstloss-trnloss)
    gap.append(tstloss-trnloss)

trn, tst = utils.get_cifar100('CNN')    
for z in [vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]:
    trnloss, trnacc = z.evaluate(trn.X, trn.Y, verbose=0)
    tstloss, tstacc = z.evaluate(tst.X, tst.Y, verbose=0)
    print(tstloss-trnloss)
    gap.append(tstloss-trnloss)

print(gap)

4.251395093300343
3.7546866759109494
2.549686437034607
1.3878222824859618
1.7264234423065186
5.7566505273723605
5.660707189331054
4.320865329372883
4.583720909181331
[4.251395093300343, 3.7546866759109494, 2.549686437034607, 1.3878222824859618, 1.7264234423065186, 5.7566505273723605, 5.660707189331054, 4.320865329372883, 4.583720909181331]


In [3]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('PFN:')

PFN_measure = []
for z in all_final:
    wf = z.get_weights()
    pp = 1
    for w in wf:
        if len(w.shape)>1:
            pp = pp*LA.norm(w) 
    PFN_measure.append(pp)
print(PFN_measure)

PFN:
[224352787.53549945, 100377646.76324835, 1428014975680.7751, 6.630076711849133e+16, 2.0645483419625482e+20, 1.0902068882426622e+24, 1.563125939975015e+39, 9.134087724244897e+76, 1.4347570152667757e+192]


In [4]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('PSN:')

PSN_measure = []
for z in all_final:
    pp = 1
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            if wf[i].shape[0] > wf[i].shape[1]:
                aa = wf[i].T.dot(wf[i])
            else:
                aa = wf[i].dot(wf[i].T)     
            pp = pp * np.sqrt(np.max(np.sum(np.abs(aa),axis=1)))
        elif len(wf[i].shape) == 4:
            aa = 0
            for w1 in wf[i]:
                for w2 in w1:
                    zz = w2.dot(w2.T)
                    aa = aa + np.max(np.sum(np.abs(zz),axis=1))
            pp = pp * np.sqrt(aa)
    PSN_measure.append(pp)
print(PSN_measure)    

PSN:
[20672.27287672647, 14853.760327674816, 3591710.59497501, 443729345.7899164, 15829121228.844782, 4569526074501.898, 9.173079117049581e+18, 1.0655301834236763e+46, 4.264056339718124e+153]


In [5]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('NOP:')

NOP_measure = []
for z in all_final:
    pp = 1
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            pp = pp + wf[i].shape[0]*wf[i].shape[1]
        elif len(wf[i].shape) == 4:
            pp = pp + wf[i].shape[0]*wf[i].shape[1]*wf[i].shape[2]*wf[i].shape[3]
    NOP_measure.append(pp)
print(NOP_measure)  

NOP:
[37360001, 42820001, 9793217, 15285953, 20594369, 11222209, 21322945, 23659713, 6972609]


In [6]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]
all_initial = [fcn1_initial, fcn2_initial,vgg11_initial, vgg16_initial, vgg19_initial, resnet18_initial, resnet34_initial, resnet50_initial, densenet121_initial] 
print('SOSP:')

SOSP_measure = []
for z,x in zip(all_final,all_initial):
    n=0
    pp=0
    wf = z.get_weights()
    w0 = x.get_weights()
    
    for i in range(len(wf)):
        wf[i]=wf[i]-w0[i]
        if len(wf[i].shape) == 2:
            n = n + wf[i].shape[0]*wf[i].shape[1]
        elif len(wf[i].shape) == 4:
            n = n + wf[i].shape[0]*wf[i].shape[1]*wf[i].shape[2]*wf[i].shape[3]

        if len(wf[i].shape) == 2:
            if wf[i].shape[0] > wf[i].shape[1]:
                aa = wf[i].T.dot(wf[i])
            else:
                aa = wf[i].dot(wf[i].T)     
            pp = pp + np.sqrt(np.max(np.sum(np.abs(aa),axis=1)))
        elif len(wf[i].shape) == 4:
            aa = 0
            for w1 in wf[i]:
                for w2 in w1:
                    zz = w2.dot(w2.T)
                    aa = aa + np.max(np.sum(np.abs(zz),axis=1))
            pp = pp + np.sqrt(aa)
    pp = n*pp
    SOSP_measure.append(pp)
print(SOSP_measure) 

SOSP:
[870619559.3070984, 984466756.310463, 427605635.84409696, 699868760.9188745, 1086421145.9780598, 867718324.2066714, 3099976133.037247, 5212987345.779156, 16502330410.619022]


In [7]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('WC:')

WC_measure = []
for z in all_final:
    pp = 1
    n=0
    counter = 0
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            n = n + np.mean(np.abs(cosine(wf[i][:9])))
            counter += 1
        elif len(wf[i].shape) == 4 and wf[i].shape[0]>1:
            wf[i] = wf[i].transpose((2,3,0,1))
            wf[i] = wf[i].reshape(wf[i].shape[0],wf[i].shape[1],-1)
            for zz in wf[i]:
                n = n + np.mean(np.abs(cosine(zz.T)))
                counter += 1
    WC_measure.append(n/counter)
print(WC_measure)   

WC:
[0.27586371302604673, 0.2781343460083008, 0.28092431358641423, 0.27954561825623236, 0.27771727591097095, 0.764596920965066, 0.7810627145878237, 0.27886186537761537, 0.38944591411104046]


In [8]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]
all_initial = [fcn1_initial, fcn2_initial,vgg11_initial, vgg16_initial, vgg19_initial, resnet18_initial, resnet34_initial, resnet50_initial, densenet121_initial] 
print('PB:')

PB_measure = []
for z,x in zip(all_final,all_initial):
    pp=0
    wf = z.get_weights()
    w0 = x.get_weights()
    
    for i in range(len(wf)):
        pp = pp+LA.norm(wf[i]-w0[i])**2
    PB_measure.append(pp/2) #assume 2*sigma^2 = 2
print(PB_measure)

PB:
[499.7317981586018, 489.3143142363713, 5219.481303902085, 3897.7522113471214, 5225.044258124161, 5358.507097778803, 10638.252448871905, 1136224.3536500433, 1934697257.2833683]


In [12]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('gw:')

gw_measure = []
for z in all_final:
    counter = 0
    n=0
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            zz = np.mean(np.abs(cosine(wf[i])))
            n = n - (wf[i].shape[1]-1)*wf[i].shape[0]*np.log(1-zz)-wf[i].shape[0]*np.log(1+wf[i].shape[1]*zz-zz)
            counter += 1
        elif len(wf[i].shape) == 4 and wf[i].shape[0]>1:
            n_in = wf[i].shape[2]
            n_out = wf[i].shape[3]

            wf[i] = wf[i].transpose((2,3,0,1))
            wf[i] = wf[i].reshape(wf[i].shape[0],wf[i].shape[1],-1)
            zz = 0
            for xx in wf[i]:
                zz = zz + np.mean(np.abs(cosine(xx.T)))/len(wf[i])
            if zz < 1:    
                zzz = (n_out-1)*n_in*np.log(1-zz)-n_in*np.log(1+n_out*zz-zz)
            else:
                zzz = -50000
            n = n - zzz
            counter += 1
    gw_measure.append(n/counter)
print(gw_measure) 

gw:
[110826.08734453183, 127056.93950873404, 33665.21304855671, 35697.00304765269, 40184.34656449328, 153976.38975235666, 151922.462288232, 24774.73840998059, 2383.1638428583424]


In [14]:
import numpy as np
def kendall(a,b):
    pos = 0
    neg = 0
    for i in range(len(a)-1):
        for j in range(i+1,len(a)):
            if (a[i]-a[j])*(b[i]-b[j])>0:
                pos += 1
            else:
                neg += 1
    print('POS:',pos)
    print('NEG:',neg)
    print((pos-neg)/(pos+neg))
    
print('-----PFN-----')    
kendall(gap,np.array(PFN_measure)) 

print('-----PSN-----')  
kendall(gap,np.array(PSN_measure)) 

print('-----NOP-----')  
kendall(gap,np.array(NOP_measure)) 

print('-----SOSP-----')  
kendall(gap,np.array(SOSP_measure)) 

print('-----WC-----')  
kendall(gap,np.array(WC_measure)) 

print('-----PB-----')  
kendall(gap,np.array(PB_measure)) 

print('-----gw-----') 
kendall(gap,np.array(gw_measure)) 

print('-----PBC-----') 
kendall(gap,np.array(PB_measure)+np.array(gw_measure)) 

-----PFN-----
POS: 23
NEG: 13
0.2777777777777778
-----PSN-----
POS: 23
NEG: 13
0.2777777777777778
-----NOP-----
POS: 16
NEG: 20
-0.1111111111111111
-----SOSP-----
POS: 23
NEG: 13
0.2777777777777778
-----WC-----
POS: 26
NEG: 10
0.4444444444444444
-----PB-----
POS: 24
NEG: 12
0.3333333333333333
-----gw-----
POS: 22
NEG: 14
0.2222222222222222
-----PBC-----
POS: 28
NEG: 8
0.5555555555555556
